In [1]:
import cv2
import numpy as np
VIDEO_PATH = "volleyball_match.mp4"


In [2]:
BALL_COLOR_RANGES = [
    (np.array([10, 100, 120]), np.array([25, 255, 255])),
    (np.array([3, 100, 100]), np.array([10, 255, 255]))
]
RED_RANGES = [
    (np.array([0, 150, 120]), np.array([10, 255, 255])),
    (np.array([160, 150, 120]), np.array([179, 255, 255])) 
]
YELLOW_RANGES = [
    (np.array([18, 80, 100]), np.array([35, 255, 255])),
    (np.array([10, 40, 40]), np.array([25, 200, 180]))
]
BLUE_RANGE = (np.array([100, 100, 50]), np.array([130, 255, 255])) 
FLOOR_MASKS = [
    (np.array([2, 180, 180]), np.array([8, 255, 255])), 
    (np.array([160, 100, 140]), np.array([180, 230, 255])) 
]
NET_MASKS = [
    (np.array([0, 0, 200]), np.array([180, 30, 255]))
]



In [3]:
def apply_clahe_to_v(hsv):
    h, s, v = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    v = clahe.apply(v)
    return cv2.merge((h, s, v))



In [4]:
def mask_from_ranges(hsv, ranges):
    mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
    for lower, upper in ranges:
        mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower, upper))
    return mask



In [5]:
def filter_floor_and_net(mask, hsv):
    for lower, upper in FLOOR_MASKS + NET_MASKS:
        floor_mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.bitwise_and(mask, cv2.bitwise_not(floor_mask))
    return mask



In [6]:
def get_player_boxes(hsv_roi, w, h):
    player_boxes = []
    mask_red = np.zeros(hsv_roi.shape[:2], dtype=np.uint8)
    for lower, upper in RED_RANGES:
        mask_red = cv2.bitwise_or(mask_red, cv2.inRange(hsv_roi, lower, upper))
    mask_red = filter_floor_and_net(mask_red, hsv_roi)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask_red = cv2.dilate(mask_red, np.ones((6, 6), np.uint8), iterations=2)
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours_red:
        x_, y_, w_, h_ = cv2.boundingRect(cnt)
        area = w_ * h_
        aspect_ratio = h_ / (w_ + 1e-5)
        if h_ < 40 or w_ < 20 or area < 1200 or area > 25000 or aspect_ratio < 1.1 or aspect_ratio > 7.0:
            continue
        if y_ > hsv_roi.shape[0] * 0.8:
            continue
        roi = hsv_roi[y_:y_+h_, x_:x_+w_]
        red_pixels = cv2.countNonZero(mask_from_ranges(roi, RED_RANGES))
        total_pixels = roi.shape[0] * roi.shape[1]
        if total_pixels == 0 or (red_pixels / total_pixels) < 0.08:
            continue

        roi_hsv = hsv_roi[y_:y_+h_, x_:x_+w_]
        blue_mask = cv2.inRange(roi_hsv, np.array([100, 80, 40]), np.array([130, 255, 200]))
        white_mask = cv2.inRange(roi_hsv, np.array([0, 0, 200]), np.array([180, 40, 255]))

        blue_count = cv2.countNonZero(blue_mask)
        white_count = cv2.countNonZero(white_mask)
        if total_pixels > 0 and (blue_count / total_pixels) > 0.08 and (white_count / total_pixels) > 0.08:
            x = x_ + int(w * 0.1)
            y = y_ + int(h * 0.2)
            player_boxes.append((x, y, w_, h_, "red"))
            continue

        x = x_ + int(w * 0.1)
        y = y_ + int(h * 0.2)
        player_boxes.append((x, y, w_, h_, "red"))

    mask_yellow = mask_from_ranges(hsv_roi, YELLOW_RANGES)
    mask_blue = cv2.inRange(hsv_roi, BLUE_RANGE[0], BLUE_RANGE[1])
    mask_yellow = cv2.bitwise_or(mask_yellow, mask_blue)
    mask_yellow = filter_floor_and_net(mask_yellow, hsv_roi)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask_yellow = cv2.dilate(mask_yellow, np.ones((6, 6), np.uint8), iterations=2)
    contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours_yellow:
        x_, y_, w_, h_ = cv2.boundingRect(cnt)
        area = w_ * h_
        aspect_ratio = h_ / (w_ + 1e-5)
        if h_ < 40 or w_ < 20 or area < 1200 or area > 25000 or aspect_ratio < 1.1 or aspect_ratio > 7.0:
            continue
        x = x_ + int(w * 0.1)
        y = y_ + int(h * 0.2)
        player_boxes.append((x, y, w_, h_, "yellow"))
    return player_boxes



In [7]:
def overlap_with_players(x, y, w, h, player_boxes):
    for px, py, pw, ph, _ in player_boxes:
        if (x + w//2 > px and x + w//2 < px + pw and
            y + h//2 > py and y + h//2 < py + ph):
            return True
    return False



In [8]:
def detect_ball(frame, hsv, motion_mask, player_boxes):
    best_ball = None
    best_score = 0
    ball_mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
    for lower, upper in BALL_COLOR_RANGES:
        ball_mask = cv2.bitwise_or(ball_mask, cv2.inRange(hsv, lower, upper))
    ball_mask = filter_floor_and_net(ball_mask, hsv)
    ball_motion = cv2.bitwise_and(ball_mask, motion_mask)
    ball_motion = cv2.morphologyEx(ball_motion, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    ball_motion = cv2.morphologyEx(ball_motion, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    contours, _ = cv2.findContours(ball_motion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if 10 < area < 350:
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            x, y, w_, h_ = cv2.boundingRect(contour)
            if circularity > 0.7 and not overlap_with_players(x, y, w_, h_, player_boxes):
                score = circularity * area
                if score > best_score:
                    best_score = score
                    best_ball = (x, y, w_, h_)
    if best_ball:
        x, y, w_, h_ = best_ball
        cv2.circle(frame, (x + w_ // 2, y + h_ // 2), max(w_, h_) // 2, (0, 255, 0), 3)
        cv2.putText(frame, "BALL", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 3)



In [9]:
def main():
    capture = cv2.VideoCapture(VIDEO_PATH)
    ret, prev_frame = capture.read()
    prev_frame = cv2.resize(prev_frame, (960, 540))
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    max_red_count = 0
    max_yellow_count = 0

    while True:
        ret, frame = capture.read()
        
        if not ret:
            break
        frame = cv2.resize(frame, (960, 540))
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hsv = apply_clahe_to_v(hsv)
        h, w, _ = frame.shape
        roi = frame[int(h * 0.2):int(h * 0.9), int(w * 0.1):int(w * 0.9)]
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        hsv_roi = apply_clahe_to_v(hsv_roi)

        player_boxes = get_player_boxes(hsv_roi, w, h)
        team_counts = {"red": 0, "yellow": 0}

        for x, y, w_, h_, team in player_boxes:
            color = (0, 0, 255) if team == "red" else (0, 255, 255)
            cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
            cv2.putText(frame, team.capitalize(), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            team_counts[team] += 1

        max_red_count = max(max_red_count, team_counts["red"])
        max_yellow_count = max(max_yellow_count, team_counts["yellow"])

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_diff = cv2.absdiff(prev_gray, gray)
        _, motion_mask = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
        detect_ball(frame, hsv, motion_mask, player_boxes)
        prev_gray = gray.copy()

        cv2.putText(frame, f"Red: {team_counts['red']} (Max: {max_red_count})", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.putText(frame, f"Yellow: {team_counts['yellow']} (Max: {max_yellow_count})", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        cv2.imshow("Volleyball Match Analysis", frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
        
    capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()